# Import the required libraries

In [1]:
import pandas as pd
import numpy as np

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spaCy
import spacy
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#vis 
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

C:\ProgramData\Anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# Import data

In [2]:
imdb_data = pd.read_csv("imdb_labelled.txt",  names=['review','label'], delimiter=".\t")
imdb_data

C:\Users\Quynh Pham\AppData\Local\Temp\ipykernel_12432\3627405509.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  imdb_data = pd.read_csv("imdb_labelled.txt",  names=['review','label'], delimiter=".\t")


,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [3]:
review = imdb_data.review.tolist()
review[:10]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man. ',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out. ',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent. ',
 'Very little music or anything to speak of. ',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head. ',
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty. ",
 'Wasted two hours. ',
 'Saw the movie today and thought it was a good effort, good messages for kids. ',
 'A bit predictable. ',
 'Loved the casting of Jimmy Buffet as the science teacher. ']

# Prepare data

In [4]:
# create a spacy object, disable parser and ner for the script to run a bit faster
nlp = spacy.load('en_core_web_sm', disable=['parser','ner']) 

#get the list of stop words
stopwords = stopwords.words('english')
stopwords.append('movie')
stopwords.append('film')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# function to preprocess the review
def preprocess(texts, allowed_postags=['NOUN','VERB']):    
    lemmatizer = WordNetLemmatizer()
    filter_sentence = []
    temp = []
    words = gensim.utils.simple_preprocess(str(texts), deacc=True) # gensim - lowercase, tokenize, de-accents
    words = [w for w in words if not w in stopwords] # stopwords removal
    temp = ' '.join(x for x in words) # combine the tokens into a sentence again
    doc = nlp(temp)
    for word in doc: # go through each words in that clean sentence
        if word.pos_ in allowed_postags: # remove words that are not verb or noun
            filter_sentence.append(word.lemma_) # lemmatization
    return filter_sentence

In [6]:
# build the dictionary
corpus = [preprocess(line) for line in review]
corpus[:3]

[['move', 'drift', 'man'],
 ['lose', 'character', 'audience', 'half', 'walk'],
 ['attempt',
  'artiness',
  'camera',
  'angle',
  'disappoint',
  'become',
  'act',
  'plot',
  'line',
  'existent']]

In [7]:
# build the dicationary with gensim
dictionary = corpora.Dictionary(corpus)
len(dictionary)

1435

In [8]:
# convert corpus into bag-of-words format
bow = [dictionary.doc2bow(line) for line in corpus]
print(bow[0][0:20])

[(0, 1), (1, 1), (2, 1)]


# Build model with 2 topics

In [9]:
lda_model_1 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=2, passes=8, alpha="auto")

In [10]:
lda_model_1.show_topics(formatted=True, num_topics=2, num_words=20)

[(0,
  '0.025*"see" + 0.013*"make" + 0.012*"scene" + 0.010*"think" + 0.010*"go" + 0.009*"script" + 0.008*"movie" + 0.008*"give" + 0.007*"film" + 0.007*"end" + 0.007*"know" + 0.007*"find" + 0.006*"recommend" + 0.006*"line" + 0.006*"love" + 0.006*"plot" + 0.006*"thing" + 0.005*"people" + 0.005*"suck" + 0.005*"cast"'),
 (1,
  '0.018*"character" + 0.015*"time" + 0.015*"watch" + 0.013*"act" + 0.010*"make" + 0.010*"get" + 0.010*"work" + 0.007*"play" + 0.007*"way" + 0.007*"show" + 0.007*"waste" + 0.007*"story" + 0.007*"love" + 0.006*"actor" + 0.006*"look" + 0.006*"write" + 0.006*"performance" + 0.005*"screen" + 0.005*"feel" + 0.005*"see"')]

In [11]:
# Visualize model with 2 topics
pyLDAvis.enable_notebook() #allow us to see in jupyter notebook
vis1 = pyLDAvis.gensim_models.prepare(lda_model_1, bow, dictionary)
vis1

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.081817  0.0       1        1  53.587999
0     -0.081817  0.0       2        1  46.412001, topic_info=          Term       Freq      Total Category  logprob  loglift
40         see  57.000000  57.000000  Default  30.0000  30.0000
4    character  43.000000  43.000000  Default  29.0000  29.0000
25       scene  24.000000  24.000000  Default  28.0000  28.0000
34        work  20.000000  20.000000  Default  27.0000  27.0000
105       time  35.000000  35.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
45        love  10.757652  24.991780   Topic2  -5.1415  -0.0753
48        look   9.374555  22.055458   Topic2  -5.2791  -0.0879
139      story   9.105721  24.301747   Topic2  -5.3082  -0.2140
176      actor   8.106896  21.352485   Topic2  -5.4244  -0.2008
193   dialogue   6.466299  10.221796   Topic2  -5.6505   0.3097

[140 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
8         1  0.839170     act
8         2  0.179822     act
485       1  0.153539  action
485       2  0.921236  action
176       1  0.608828   actor
...     ...       ...     ...
157       1  0.798421   write
157       2  0.245668   write
643       1  0.807493  writer
223       1  0.342306    year
223       2  0.616151    year

[166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

# Build model with 5 topics

In [12]:
lda_model_2 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=5, passes=8, alpha="auto")

In [13]:
lda_model_2.show_topics(formatted=True, num_topics=5, num_words=20)

[(0,
  '0.012*"act" + 0.010*"scene" + 0.010*"part" + 0.010*"play" + 0.010*"year" + 0.007*"leave" + 0.006*"comedy" + 0.006*"fan" + 0.006*"way" + 0.006*"fail" + 0.005*"watch" + 0.005*"story" + 0.005*"drama" + 0.005*"scamp" + 0.005*"sound" + 0.005*"death" + 0.005*"know" + 0.005*"rating" + 0.005*"dislike" + 0.005*"suspense"'),
 (1,
  '0.027*"watch" + 0.026*"time" + 0.021*"love" + 0.016*"character" + 0.014*"line" + 0.014*"script" + 0.013*"waste" + 0.011*"cast" + 0.010*"enjoy" + 0.010*"story" + 0.007*"make" + 0.007*"performance" + 0.006*"way" + 0.006*"see" + 0.006*"look" + 0.006*"show" + 0.006*"write" + 0.006*"know" + 0.006*"game" + 0.005*"level"'),
 (2,
  '0.021*"get" + 0.014*"see" + 0.013*"make" + 0.012*"plot" + 0.012*"thing" + 0.011*"character" + 0.009*"work" + 0.008*"scene" + 0.007*"script" + 0.007*"director" + 0.007*"show" + 0.006*"say" + 0.006*"actor" + 0.006*"play" + 0.006*"watch" + 0.006*"seem" + 0.006*"garbage" + 0.005*"act" + 0.005*"world" + 0.005*"place"'),
 (3,
  '0.019*"act" + 0

In [14]:
# Visualize model with 5 topics
vis2 = pyLDAvis.gensim_models.prepare(lda_model_2, bow, dictionary)
vis2

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.047301  0.116734       1        1  24.809779
1     -0.126108 -0.050870       2        1  21.178555
3      0.090311 -0.008472       3        1  19.746166
2      0.058641  0.037308       4        1  18.812502
0      0.024457 -0.094699       5        1  15.452999, topic_info=      Term       Freq      Total Category  logprob  loglift
207  movie  22.000000  22.000000  Default  30.0000  30.0000
45    love  25.000000  25.000000  Default  29.0000  29.0000
105   time  35.000000  35.000000  Default  28.0000  28.0000
350   suck  12.000000  12.000000  Default  27.0000  27.0000
311    get  23.000000  23.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
139  story   3.030615  24.453529   Topic5  -5.3086  -0.2206
181   feel   2.615743  10.986360   Topic5  -5.4558   0.4323
48    look   2.683323  22.334822   Topic5  -5.4303  -0.2517
40     see   2.262315  59.249642   Topic5  -5.6010  -1.3980
311    get   2.195735  23.243943   Topic5  -5.6309  -0.4922

[327 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1149      4  0.918548  achievement
8         1  0.092114          act
8         2  0.092114          act
8         3  0.460571          act
8         4  0.122819          act
...     ...       ...          ...
223       1  0.344638         year
223       2  0.137855         year
223       3  0.068928         year
223       4  0.068928         year
223       5  0.413565         year

[518 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 3, 1])

# Build model with 10 topics

In [15]:
lda_model_3 = gensim.models.LdaModel(corpus=bow, id2word=dictionary, num_topics=10, passes=8, alpha="auto")

In [16]:
lda_model_3.show_topics(formatted=True, num_topics=10, num_words=20)

[(0,
  '0.029*"give" + 0.028*"character" + 0.026*"make" + 0.022*"actor" + 0.020*"cast" + 0.019*"performance" + 0.018*"watch" + 0.017*"play" + 0.012*"job" + 0.012*"feel" + 0.012*"line" + 0.010*"story" + 0.009*"think" + 0.009*"show" + 0.007*"plot" + 0.007*"time" + 0.007*"scamp" + 0.007*"go" + 0.007*"deliver" + 0.006*"script"'),
 (1,
  '0.033*"end" + 0.026*"act" + 0.017*"go" + 0.015*"think" + 0.012*"see" + 0.012*"make" + 0.011*"music" + 0.010*"people" + 0.010*"sound" + 0.010*"time" + 0.009*"movie" + 0.009*"find" + 0.008*"love" + 0.008*"write" + 0.008*"death" + 0.007*"budget" + 0.007*"scene" + 0.007*"live" + 0.006*"experience" + 0.006*"portrayal"'),
 (2,
  '0.024*"make" + 0.019*"screen" + 0.016*"character" + 0.015*"plot" + 0.012*"scene" + 0.012*"see" + 0.012*"act" + 0.011*"say" + 0.011*"use" + 0.010*"get" + 0.010*"lack" + 0.009*"actor" + 0.008*"way" + 0.008*"write" + 0.007*"work" + 0.007*"time" + 0.007*"cinematography" + 0.006*"cartoon" + 0.006*"fact" + 0.006*"go"'),
 (3,
  '0.045*"see" + 

In [17]:
# Visualize model with 10 topics
vis3 = pyLDAvis.gensim_models.prepare(lda_model_3, bow, dictionary)
vis3

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.075449  0.042635       1        1  13.571369
9      0.072526  0.122823       2        1  11.657377
0      0.105113  0.010689       3        1  11.060655
3     -0.071482  0.019500       4        1  10.487807
1     -0.039064  0.029820       5        1  10.224596
5      0.024175 -0.098530       6        1   9.439972
4      0.006137 -0.139250       7        1   9.303722
7      0.039325 -0.027840       8        1   8.528677
8     -0.144786  0.047189       9        1   7.959354
6     -0.067393 -0.007036      10        1   7.766471, topic_info=          Term       Freq      Total Category  logprob  loglift
48        look  21.000000  21.000000  Default  30.0000  30.0000
350       suck  11.000000  11.000000  Default  29.0000  29.0000
165  recommend  11.000000  11.000000  Default  28.0000  28.0000
249        end  17.000000  17.000000  Default  27.0000  27.0000
207      movie  21.000000  21.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
40         see   2.124406  58.020780  Topic10  -4.9759  -0.7520
50        make   1.702946  47.277829  Topic10  -5.1970  -0.7683
41       think   1.601435  24.408240  Topic10  -5.2585  -0.1687
112       play   1.440949  19.855067  Topic10  -5.3641  -0.0678
176      actor   1.435365  21.667549  Topic10  -5.3680  -0.1590

[623 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
8         1  0.180610   act
8         2  0.090305   act
8         3  0.030102   act
8         4  0.030102   act
8         5  0.301017   act
...     ...       ...   ...
223       5  0.134969  year
223       6  0.067484  year
223       7  0.067484  year
223       8  0.067484  year
223       9  0.067484  year

[1030 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 1, 4, 2, 6, 5, 8, 9, 7])